In [1]:
# Import dependencies
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import shapely, os, copy
import osmnx as ox
import UrbanAccessAnalyzer.osm
import UrbanAccessAnalyzer.service_quality 
import UrbanAccessAnalyzer.accessibility 
import UrbanAccessAnalyzer.population
import UrbanAccessAnalyzer.graph_processing
from UrbanAccessAnalyzer.utils import ipyleaflet_drawable_map

In [2]:
m, draw_features = ipyleaflet_drawable_map(center=[48.2050,16.3701], zoom=15) # center=[lat, lon]
m

Map(center=[48.205, 16.3701], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

New geometry added: POLYGON ((16.352511 48.202562, 16.352511 48.213601, 16.382886 48.213601, 16.382886 48.202562, 16.352511 48.202562))


In [3]:
bounds = draw_features()

In [4]:
area_steps = [500,1000,5000,10000,100000]
distance_steps = [300,500,900,1250]
graph_node_cluster_distance = 10

In [5]:
pedestrian_graph = UrbanAccessAnalyzer.osm.download_graph(bounds=bounds,network_type='walk')
pedestrian_graph = UrbanAccessAnalyzer.graph_processing.cluster_nodes(pedestrian_graph,distance=graph_node_cluster_distance)
ox.save_graphml(pedestrian_graph,'pedestrian_graph.graphml')

In [6]:
green_areas = UrbanAccessAnalyzer.osm.green_areas(bounds,pedestrian_graph=pedestrian_graph)
green_areas = UrbanAccessAnalyzer.service_quality.by_area(green_areas,areas=area_steps)
green_areas.to_file('green_areas.gpkg')
green_areas

,service_type,geometry,service_quality
0,park,"POLYGON ((16.37715 48.20316, 16.37715 48.20316...",2
1,park,"POLYGON ((16.37882 48.20251, 16.37908 48.20271...",2
2,grass,"POLYGON ((16.37426 48.21283, 16.37463 48.21308...",4
3,park,"POLYGON ((16.36395 48.20456, 16.3639 48.20459,...",2
5,park,"POLYGON ((16.36454 48.21154, 16.36474 48.21169...",5
6,grass,"POLYGON ((16.36149 48.20643, 16.36137 48.20652...",3
7,grass,"POLYGON ((16.36275 48.20734, 16.36322 48.20762...",3
10,park,"POLYGON ((16.36007 48.20709, 16.36007 48.20709...",2
13,grass,"POLYGON ((16.36104 48.20499, 16.36081 48.20516...",4
14,grass,"POLYGON ((16.35869 48.2113, 16.35873 48.21146,...",2


In [7]:
accessibility_graph = UrbanAccessAnalyzer.accessibility.graph(
    green_areas,
    pedestrian_graph,
    dist_steps=distance_steps,
    undirected=False
)

accessibility quality class 1 completed
accessibility quality class 2 completed
accessibility quality class 3 completed
accessibility quality class 4 completed
accessibility quality class 5 completed
accessibility quality class 6 completed
accessibility quality class 7 completed


In [8]:
nodes,edges=ox.graph_to_gdfs(accessibility_graph)

In [9]:
m=None
m=green_areas.explore(m=m,color='green')
if len(edges[edges['accessibility'] == 999])>0:
    m=edges[edges['accessibility'] == 999].explore(m=m,color='purple')
if len(edges[edges['accessibility'] == 5])>0:
    m=edges[edges['accessibility'] == 5].explore(m=m,color='red')
if len(edges[edges['accessibility'] == 4])>0:
    m=edges[edges['accessibility'] == 4].explore(m=m,color='brown')
if len(edges[edges['accessibility'] == 3])>0:
    m=edges[edges['accessibility'] == 3].explore(m=m,color='orange')
if len(edges[edges['accessibility'] == 2])>0:
    m=edges[edges['accessibility'] == 2].explore(m=m,color='yellow')
if len(edges[edges['accessibility'] == 1])>0:
    m=edges[edges['accessibility'] == 1].explore(m=m,color='green')
m

In [10]:
edges.to_file('accesibility_graph_edges.gpkg')